In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from pathlib import Path
model = ResNet50(weights='imagenet')
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
%pip install keract
import keract
from glob import glob
import pandas as pd
# Google Drive mount removed for local execution
keyFileNameCorr = './data/LANTERN_Clinical_Data.xlsx'
import pickle
infile = open('./data/processed/allResponsesOriented_orig.pickle','rb')
new_dict = pickle.load(infile)
infile.close()
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from matplotlib import pyplot
from numpy import where
from imblearn.under_sampling import RandomUnderSampler 
from statistics import mean
from statistics import stdev
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.datasets import make_friedman1
from sklearn.decomposition import MiniBatchSparsePCA
!pip install umap-learn
import umap
from random import sample



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
stage1_address = './data/Stage1/stage1_progression.xlsx'
df_stage1 = pd.read_excel(stage1_address, dtype=str)
# df_stage1['SexGroup'].replace(['Male', 'Female'],[0, 1], inplace=True)
# df_stage1['StageCat'].replace(['Stage1-2', 'Stage3_4'],[0, 1], inplace=True)
label_column = df_stage1['Progression.1.yes..0..no'].values
df_key_filename = pd.read_excel(keyFileNameCorr, dtype=str)
file_name_column = df_stage1['File.name'].values

In [ ]:
source_folder = './data/Data/Raw/RawImages'
metal_indicator_lung = './data/ClinicalVariablesSpecial/Metals_Indicators_LineageMarkers.xlsx'
deleteExtraFeatures = 0
random_state_num = 22


allExamples = glob(source_folder+"/*/", recursive = True)
print(len(allExamples))
df = pd.read_excel (metal_indicator_lung)
channel_names = df.Indicator.values
co_marker = df.Comarker.values
num_channels = len(channel_names)
allFileNames = new_dict['allFileNames']
allResponsesOriented = new_dict['allResponsesOriented']
if deleteExtraFeatures:
  toBeDeleted = []
  for ind in range(num_channels):
    if co_marker[ind]==0:
      for j in range(2048):
        toBeDeleted.append(ind*2048+j)
  print('Number of markers removed = '+str(len(toBeDeleted)/2048)+' out of '+str(num_channels)+' channels')
  allResponsesOriented = np.delete(allResponsesOriented,np.s_[toBeDeleted],axis=1)

416


In [ ]:
used = set()
# mylist = [u'nowplaying', u'PBS', u'PBS', u'nowplaying', u'job', u'debate', u'thenandnow']
unique = [x for x in label_column if x not in used and (used.add(x) or True)]
import math
unique = [x for x in unique if not (isinstance(x, float) and math.isnan(x))]

print(unique)
y = []
X = []

target_0 = unique[0]
target_1 = unique[1]

['1', '0']


In [ ]:
label_column[0]

'1'

In [ ]:
len(file_name_column)

286

In [ ]:
X = []
y = []
def remove_end_spaces(string):
    return "".join(string.rstrip())
for i in range(len(file_name_column)):
  ind_num = [index for index, value in enumerate(allFileNames[:416]) if value == remove_end_spaces(file_name_column[i])]
  # ind_num = [index for index, value in enumerate(file_name_column[:416]) if value == remove_end_spaces(allFileNames[i])]
  # if index_num.shape[0] == 0:
  #   print()
  y_val = label_column[i]
  # print(val)

  if y_val==target_0 or y_val==target_1:    
    y.append(y_val)
    if len(X)==0:
      X = allResponsesOriented[ind_num,:]
    else:
      X = np.vstack([X, allResponsesOriented[ind_num,:]])



copyX = X
copyY = y

In [ ]:
X.shape

(286, 75776)

In [ ]:
X = copyX
y = copyY
A = Counter(y)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  n0 = int(class_0/10)
  n1 = int(class_1/10)
  baseline_dist = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  ratio = max(class_0,class_1)/min(class_0,class_1)
elif len(unique) > 2:
  ratio = 2
print('Original dataset shape %s' % Counter(y))
if ratio >=2:
  ros = RandomOverSampler(random_state=42)
  X, y = ros.fit_resample(X, y)
  print('Resampled dataset shape %s' % Counter(y))


yy = np.array(y)
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=10,shuffle=True)
kf.get_n_splits(X)

delete_dup = True
# print(kf)

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = yy[train_index], yy[test_index]
    toBeDeleted = []
    if delete_dup: 
      for i in range(X_test.shape[0]):
        query = X_test[i]
        j = 0
        query_found = 0
        while j < X_train.shape[0] and query_found==0:
          match = X_train[j]
          dist = np.linalg.norm(query-match)
          if dist == 0:
            query_found = 1
            toBeDeleted.append(i)
          j = j + 1

      if X_test.shape[0]-len(toBeDeleted) >= 5:
        X_test = np.delete(X_test,np.s_[toBeDeleted],axis=0)
        y_test = np.delete(y_test,np.s_[toBeDeleted],axis=0)
    transformer = MiniBatchSparsePCA(n_components=9, batch_size=500,random_state=0)
    transformer.fit(X_train)
    X_tr_em = transformer.transform(X_train)
    X_te_em = transformer.transform(X_test)
    kernel = 1.0 * RBF(1.0)
    clf = make_pipeline(StandardScaler(), SVC(gamma=1e1,C=1e1,kernel = kernel))
    clf.fit(X_tr_em, y_train)
    print('test score = '+str(clf.score(X_te_em,y_test)))
    
A = Counter(copyY)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  ratio = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  print('baseline = '+str(ratio))


Original dataset shape Counter({'0': 244, '1': 42})
Resampled dataset shape Counter({'1': 244, '0': 244})
test score = 0.9285714285714286
test score = 0.9607843137254902
test score = 0.9302325581395349
test score = 1.0
test score = 0.9772727272727273
baseline = 0.8531468531468531
